In [ ]:
# Import necessary libraries: numpy, scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy
import lmfit

In [ ]:
# This notebook implements a so-called "punch-out" experiment.
# When sending a signal to the readout resonator, it is possible to send too much power.

# The control electronics can output the oscillating signal with a given amplitude (power),
# which for the purposes of this example we will scale to be between 0 and 1
# Where 1 is maximum amplitude and 0 is off.

# When sending a signal to the resonator, you are injecting photons into the resonant cavity.
# These photons can interact with the nearby qubit, driving it around the bloch sphere.
# So, when you read-out at high power, your qubit could be anywhere on the bloch sphere!

# We could send our signal at very low power, so that the qubit is not affected.
# However, this would give us a very low signal-to-noise ratio.
# So this punch-out experiment determines what is the maximum amplitude readout signal we can send.

# When we operate at low amplitude, we know that the qubit is in its ground state.
# This corresponds to the |0> computational state.
# It also means that that frequency of the resonator shifts.
# This is how we can find the maximum readout amplitude!

In [ ]:
# These parameters are used for the simulation.
# They define the properties of the simulated resonator.
SIMULATION_F_0 = 5.55
SIMULATION_AMPLITUDE_LIMIT = 0.5
SIMULATION_DISPERSIVE_SHIFT = -0.002
SIMULATION_NOISY_REGION_LIMIT = 0.7
SIMULATION_Q = 10000
SIMULATION_NOISE_SCALE = 0.02

In [ ]:
# This function is a way of describing a linear resonator in terms of its Q factors and f_0.
def linear_resonator(f, f_0, Q, Q_e_real, Q_e_imag):
    Q_e = Q_e_real + 1j*Q_e_imag
    return 1 - (Q * Q_e**-1 / (1 + 2j * Q * (f - f_0) / f_0))

In [ ]:
# Here we create a ResonatorModel to simulate the resonator.
# lmfit provides some helpful functions.
class ResonatorModel(lmfit.model.Model):
    __doc__ = "resonator model" + lmfit.models.COMMON_INIT_DOC

    def __init__(self, *args, **kwargs):
        # pass in the defining equation so the user doesn't have to later
        super().__init__(linear_resonator, *args, **kwargs)

        self.set_param_hint('Q', min=0)  # enforce Q is positive

In [ ]:
def measure_simulated_resonator(measurement_axis, amplitude):
    """Measure the simulated resonator.
    
    Arguments:
      - measurement_axis: a numpy array containing a set of frequencies to measure at.
      - amplitude: A value between 0 and 1, corresponding to the power of the signal.
      
    Returns:
      - measured_s21: a numpy array containing values corresponding to the complex response of the resonator.
    """
    # Here we set up the simulation.
    # I have provided some typical values for the resonator.
    # f_0 is in GHz, whilst the Q factors are unitless.
    resonator = ResonatorModel()
    if amplitude > SIMULATION_NOISY_REGION_LIMIT:
      true_params = resonator.make_params(f_0 = SIMULATION_F_0, 
                                          Q = SIMULATION_Q, 
                                          Q_e_real = SIMULATION_Q*0.9, 
                                          Q_e_imag = -SIMULATION_Q*0.9)
    elif amplitude > SIMULATION_AMPLITUDE_LIMIT:
      rng_generator = np.random.default_rng()
      rng = rng_generator.random()
      print(rng)
      if rng > 0.5:
        true_params = resonator.make_params(f_0 = SIMULATION_F_0, 
                                            Q = SIMULATION_Q, 
                                            Q_e_real = SIMULATION_Q*0.9, 
                                            Q_e_imag = -SIMULATION_Q*0.9)
      else:
        true_params = resonator.make_params(f_0 = SIMULATION_F_0 + SIMULATION_DISPERSIVE_SHIFT, 
                                            Q = SIMULATION_Q, 
                                            Q_e_real = SIMULATION_Q*0.9, 
                                            Q_e_imag = -SIMULATION_Q*0.9)
    else:
      true_params = resonator.make_params(f_0 = SIMULATION_F_0 + SIMULATION_DISPERSIVE_SHIFT, 
                                          Q = SIMULATION_Q, 
                                          Q_e_real = SIMULATION_Q*0.9, 
                                          Q_e_imag = -SIMULATION_Q*0.9)

    true_s21 = resonator.eval(params=true_params, f=measurement_axis)
    noise_scale = SIMULATION_NOISE_SCALE
    np.random.seed(123)
    measured_s21 = true_s21 + noise_scale*(np.random.randn(len(measurement_axis)) + 1j*np.random.randn(len(measurement_axis)))
    return measured_s21

In [ ]:
# For this experiment, we are going to set an amplitude value, alongside our frequency sweep.
# You can use the same values of `start`, `stop`, and `num_points` that you used in
# the resonator spectroscopy experiment.

start = 5.0 # GHz
stop = 6.0 # GHz
num_points = 100

# Create a numpy array that holds these values
measurement_axis = np.linspace(start, stop, num_points)

# We also set an amplitude. Try changing this!
amplitude = 1

# Now, we simulate sending signals to the resonator at this amplitude.
# For each measurement point, we change the frequency, but the amplitude remains the same.
# In the Resonator Spectroscopy experiment, the amplitude was 1 by default.
# So the following graph should look the same.

# We simulate the experiment, and now provide `amplitude` as well as `measurement_axis`. 
response = measure_simulated_resonator(measurement_axis, amplitude)

print(response)

In [ ]:
# You will notice that the response is complex. You can use the following `numpy` functions to access the different values:

I = response.real
print("I: %s" %I)
# The real part of the complex signal corresponds to the in-phase component of the return wave, or the "I" value.

Q = response.imag
print("Q: %s" %Q)
# The imaginary part of the signal corresponds to the quadrature component of the return wave, or the "Q" value.

Magnitude = np.abs(response)
print("Magnitude: %s" %Magnitude)
# Calculating the magnitude of the signal is as simple as taking the absolute value.

In [ ]:
# We can visualise the response by plotting the data against the measurement axis.

# Note that here we convert the response to decibels by multiplying by `20*np.log10`.
plt.plot(measurement_axis, 20*np.log10(Magnitude))

plt.ylabel('Signal (dB)')
plt.xlabel('Frequency (GHz)')
plt.title(f'{amplitude=}: Magnitude')

plt.figure()
plt.plot(measurement_axis, I)
plt.ylabel('Signal (a.u.)')
plt.xlabel('Frequency (GHz)')
plt.title(f'{amplitude=}: I')

plt.figure()
plt.plot(measurement_axis, Q)
plt.ylabel('Signal (a.u.)')
plt.xlabel('Frequency (GHz)')
plt.title(f'{amplitude=}: Q')

In [ ]:
# Try changing the amplitude in the code above, and see if the plots change.
# You should find that at low amplitudes, the frequency of the resonator shifts.
# This shift might be small! Try using a narrow window.

# Measure the difference between the frequency at high power and low power.
# This shift is called the `dispersive shift`, or `chi`, and its value is usually negative.
# The frequency at low power is the "readout frequency in the dispersive regime"
# "Dispersive regime" corresponds to the state where the amplitude is low,
# and the qubit remains in its ground state.

# You may also notice that there is a region where it seems random which frequency you see.
# This is a so-called unstable region, where the energy of the photons in your signal
# is roughly equivalent to the energy of the transition between the |0> and |1> states.
# So the signal you are using to read-out is actually driving the qubit into a superposition.
# This is the first quantum effect we will see. (Here it is simulated with RNG)

In [ ]:
# As a next step, instead of arbitarily selecting an amplitude, we can include the amplitude
# as a second sweep axis.

# This means that we will perform an experiment where we sweep the frequency, then change the 
# amplitude, then sweep the frequency again, until we have checked a whole range of amplitudes.

# We call this a 2-dimensional experiment, or a 2D sweep.

# Let's choose our sweep axes, using more explicit names to describe them:
frequency_start = 5.0
frequency_stop = 6.0
frequency_points = 100

frequency_axis = np.linspace(frequency_start, frequency_stop, frequency_points)

# Let's start low with the amplitude, and increase it until we see the qubit is no longer
# in the ground state: this is the "punch-out" experiment.
amplitude_start = 0
amplitude_stop = 1
amplitude_points = 100

amplitude_axis = np.linspace(amplitude_start, amplitude_stop, amplitude_points)

In [ ]:
# Now to simulate the 2D experiment, we calculate the resonator response for each amplitude value.
# We'll need to collect this data in an array.

data = []

# We loop over the values in the amplitude axis:
for amplitude in amplitude_axis:
    print(f"Measuring the resonator at {amplitude=}")
    # For each value of amplitude, let's measure the resonator.
    response = measure_simulated_resonator(frequency_axis, amplitude)
    # We add this to our data array
    data.append(response)

In [ ]:
# Now we need a smart way to visualise the data.
# One of the best ways to see 2D data is to plot a heatmap.

I = np.asarray(data).real
Q = np.asarray(data).imag
magnitude = np.abs(np.asarray(data))

plt.imshow(magnitude, 
           aspect="auto", 
           origin="lower", 
           extent=[frequency_start, frequency_stop, amplitude_start, amplitude_stop],
           interpolation = "none"
)

plt.show()

In [ ]:
# In the plot above, you should see three regions:
# One at high amplitude, where the frequency of the resonator is f_0
# One at low amplitude, where the frequency of the resonator is f_0 + chi
# One in the centre, where the resonator randomly swaps between the two.

# As a bonus exercise, can you find a way to identify the boundaries of these areas programatically?
# You should obtain something like 0.5 and 0.7 as the interfaces.
# First step would be to calculate the dispersive shift, `chi`.